## Install Flask to enable network communication (micro-services)

In [3]:
#!pip install flask

In [4]:
import pandas as pd
from model import Fraud_Detector_Model
from metrics import Metrics
import os
from data_pipeline import ETL_Pipeline

In [5]:
# This block is to generate dummy data to show model can make a prediction on a new transaction
data=pd.read_csv('transactions.csv')
#data.head(1)

data[1:9].to_csv('data_tocheck.csv', index=False)

# Transformation using same steps as for pre-processing for model training
etl_pipeline = ETL_Pipeline()

# read the given .csv file into a df
transactions_df = etl_pipeline.extract('transactions.csv')  # change this later for jupyter

# add features to signify time(hour) of transactions
transactions_df = etl_pipeline.categorize_time(transactions_df)

# add features to signify part of year for trasactions
transactions_df = etl_pipeline.categorize_year(transactions_df)

# add average transactions as a feature
transactions_df = etl_pipeline.calculate_avg_transactions(transactions_df)

# finally, remove all the unnecessary columns
transactions_df = etl_pipeline.remove_columns(transactions_df)

# save as a transformed csv file
transactions_df = etl_pipeline.load(transactions_df)

# above code spits out .csv file with is_fraud and new feature

# read the fully converted data 
transformed_data_full = pd.read_csv('transformed_data.csv')
#transformed_data_full.head()

# slice to generate a transaction that will go into model for testing
transformed_data_partial = transformed_data_full[2:3]

# get rid of label 
transformed_data_partial = transformed_data_partial.drop(columns=['is_fraud'])
#transformed_data_partial

# save as csv for testing
transformed_data_partial.to_csv('data_coming_in.csv', index=False)

/Users/mukulsherekar/Documents/CRAISYS/workspace/705.603Portfolio/FraudDetection/data_pipeline.py:118: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  frauds_by_age_group = transactions[transactions['is_fraud'] == 1].groupby('age_group').size()


In [7]:
# Create an instance of Fraud_Detector_Model
fdm = Fraud_Detector_Model()

# Assuming `logistic regression` is the model type
model_type = "logistic_regression"

#Call the model_infer method to perform inference on a dataframe
determination = fdm.model_infer(transformed_data_partial, model_type)

# Print the result
print("Determination:", determination)

Determination: Yes, It is a fraudulent transaction, take immediate action


/Users/mukulsherekar/opt/anaconda3/envs/en605645/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/mukulsherekar/opt/anaconda3/envs/en605645/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.3.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/mukulsherekar/opt/anaconda3/envs/en605645/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to 

In [8]:
# Create an instance of Fraud_Detector_Model
fdm = Fraud_Detector_Model()

# Assuming `logistic regression` is the model type
model_type = "logistic_regression"

# storing the csv file in a variable
csv_file = "data_coming_in.csv"

# Call the model_infer method to perform inference
determination = fdm.model_infer2(csv_file, model_type)

# # Print the result
print("Determination:", determination)

Determination: Yes, It is a fraudulent transaction, take immediate action


/Users/mukulsherekar/opt/anaconda3/envs/en605645/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/mukulsherekar/opt/anaconda3/envs/en605645/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.3.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/mukulsherekar/opt/anaconda3/envs/en605645/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to 

### Start up the service

In [ ]:
!python fraud_detection_service.py


starting server...
 * Serving Flask app 'fraud_detection_service'
 * Debug mode: on
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8786
 * Running on http://192.168.0.5:8786
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
starting server...
 * Debugger is active!
 * Debugger PIN: 504-008-368
127.0.0.1 - - [28/Feb/2024 21:08:54] "GET /infer?data_coming_in=data_coming_in.csv HTTP/1.1" 404 -
127.0.0.1 - - [28/Feb/2024 21:08:59] "GET / HTTP/1.1" 200 -
192.168.0.5 - - [28/Feb/2024 21:09:02] "GET / HTTP/1.1" 200 -


Try out the links 
* [stats](http://localhost:8786/stats)
* [determination](http://localhost:8786/infer?data_coming_in=data_coming_in.csv)

Run the following command in terminal 

wget -qO- "http://localhost:8786/infer?data_coming_in=data_coming_in.csv" /dev/null 2>&1

### You must kill the kernel to try again for the port stays locked to the current kernel